In [82]:
import gym
import minihack
from nle import nethack
from gym import spaces
import torch.nn as nn
import numpy as np
import torch
from collections import deque

In [60]:
env = gym.make('MiniHack-Quest-Hard-v0', 
               observation_keys=("glyphs", "chars", "colors", "specials",
                                 "glyphs_crop", "chars_crop", "colors_crop", "specials_crop",
                                 "blstats", "message", "inv_letters",
                                 "inv_strs", "inv_glyphs", "inv_oclasses"), 
              )
env.reset()
state, reward, done, info = env.step(0)
env.render()


                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
  #                                                                             
  @                                                                             
  <                                                                             
  #                                                                             
                                                                                
                                                                                
                           

In [61]:
print(env.action_space)

Discrete(78)


In [62]:
for i in state.keys():
    print(i)
    print(state[i].shape)
    print(state[i])

glyphs
(21, 79)
[[2359 2359 2359 ... 2359 2359 2359]
 [2359 2359 2359 ... 2359 2359 2359]
 [2359 2359 2359 ... 2359 2359 2359]
 ...
 [2359 2359 2359 ... 2359 2359 2359]
 [2359 2359 2359 ... 2359 2359 2359]
 [2359 2359 2359 ... 2359 2359 2359]]
chars
(21, 79)
[[32 32 32 ... 32 32 32]
 [32 32 32 ... 32 32 32]
 [32 32 32 ... 32 32 32]
 ...
 [32 32 32 ... 32 32 32]
 [32 32 32 ... 32 32 32]
 [32 32 32 ... 32 32 32]]
colors
(21, 79)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
specials
(21, 79)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
glyphs_crop
(9, 9)
[[   0    0 2359 2359 2359 2359 2359 2359 2359]
 [   0    0 2359 2359 2359 2359 2359 2359 2359]
 [   0    0 2359 2359 2359 2359 2359 2359 2359]
 [   0    0 2359 2359 2380 2359 2359 2359 2359]
 [   0    0 2359 2359  329 2359 2359 2359 2359]
 [   0    0 2359 2359 2382 2359 2359 2359 2359]
 [  

## CNN 1: 9 x 9 crop

In [147]:
crop_obs = ["glyphs_crop", "chars_crop", "colors_crop", "specials_crop"]
crop_state = deque([], maxlen = len(crop_obs))
for i in crop_obs:
    crop_state.append(state[i])
crop_state = torch.tensor(np.array(crop_state), dtype=torch.float).unsqueeze(0)
print(crop_state.shape)

torch.Size([1, 4, 9, 9])


In [153]:



class CNN_1(nn.Module):
    def __init__(self, observation_space, action_space):
        super().__init__()
        observation_space = torch.squeeze(observation_space)
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(observation_space.shape[0], 16, 3, stride=1, padding=1), #channels, outgoing_filters, kernerl_size, stride/no_of_frames
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),   
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
        )

        self.input_dim = self.conv_layers(torch.zeros(1, *observation_space.shape)).view(1, -1).size(1)
        #print(self.input_dim)

        self.conn_layers = nn.Sequential(
            nn.Linear(self.input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, action_space.n)
        )

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.conn_layers(x)
        return x
    
model = CNN_1(crop_state, env.action_space)
x = model.forward(crop_state.to(model.device))
print(x.shape)

observation_space torch.Size([4, 9, 9])
1296
torch.Size([1, 78])


## CNN 2: Whole State

In [155]:
obs = ["glyphs", "chars", "colors", "specials"]
whole_state = deque([], maxlen = len(obs))
for i in obs:
    whole_state.append(state[i])
whole_state = torch.tensor(np.array(whole_state), dtype=torch.float).unsqueeze(0)
print(whole_state.shape)

torch.Size([1, 4, 21, 79])


In [171]:
class CNN_2(nn.Module):
    def __init__(self, observation_space, action_space):
        super().__init__()
        observation_space = torch.squeeze(observation_space)

        self.conv_layers = nn.Sequential(
            nn.Conv2d(observation_space.shape[0], 16, 3, stride=4, padding=1), #channels, outgoing_filters, kernerl_size, stride/no_of_frames
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),   
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
        )

        self.input_dim = self.conv_layers(torch.zeros(1, *observation_space.shape)).view(1, -1).size(1)
        #print(self.input_dim)

        self.conn_layers = nn.Sequential(
            nn.Linear(self.input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, action_space.n)
        )

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.conn_layers(x)
        return x
    
model_2 = CNN_2(whole_state, env.action_space)
x_2 = model_2.forward(whole_state.to(model_2.device))
print(x_2.shape)

torch.Size([1, 78])


## CNN 3: Inventory

In [172]:
inv_obs = ["inv_strs"]
inv_state = deque([], maxlen = len(obs))
for i in inv_obs:
    #print(i)
    #print(state[i].shape)
    #print(state[i])
    inv_state.append(state[i])
inv_state = torch.tensor(np.array(inv_state), dtype=torch.float).unsqueeze(0)
print(inv_state.shape)

torch.Size([1, 1, 55, 80])


In [176]:
class CNN_3(nn.Module):
    def __init__(self, observation_space, action_space):
        super().__init__()
        observation_space = torch.squeeze(observation_space, 1)

        self.conv_layers = nn.Sequential(
            nn.Conv2d(observation_space.shape[0], 16, 3, stride=4, padding=1), #channels, outgoing_filters, kernerl_size, stride/no_of_frames
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),   
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
        )

        #print(self.conv_layers)
        self.input_dim = self.conv_layers(torch.zeros(1, *observation_space.shape)).view(1, -1).size(1)
        print(self.input_dim)
        #self.input_dim = self.get_dims(observation_space)

        self.conn_layers = nn.Sequential(
            nn.Linear(self.input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, action_space.n)
        )

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, x):
        # TODO Implement forward pass
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.conn_layers(x)
        return x
    
model_3 = CNN_3(inv_state, env.action_space)
x_3 = model_3.forward(inv_state.to(model_3.device))
print(x_3.shape)

observation_space torch.Size([1, 55, 80])
1120
torch.Size([1, 78])


## MLP 1: Stats

In [183]:
stats_obs = ["blstats", "message", "inv_letters", "inv_glyphs", "inv_oclasses"]
stats_state = np.array([])
for i in stats_obs:
    #print(i)
    #print(state[i].shape)
    #print(state[i])
    stats_state = np.append(stats_state, state[i])
stats_state = torch.tensor(stats_state, dtype=torch.float)
print(stats_state.shape)

torch.Size([447])


In [187]:
class MLP_1(nn.Module):
    def __init__(self, observation_space, action_space):
        super().__init__()
        self.input_dim = observation_space.shape[0]
        #print(self.input_dim)

        self.conn_layers = nn.Sequential(
            nn.Linear(self.input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, action_space.n)
        )

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, x):
        x = self.conn_layers(x)
        return x
    
model_4 = MLP_1(stats_state, env.action_space)
x_4 = model_4.forward(stats_state.to(model_4.device))
print(x_4.shape)

torch.Size([78])


## Joint Model:

In [236]:
class combined(nn.Module):
    def __init__(self, obs_space_crop, obs_space_whole, obs_space_inv, obs_space_stats, action_space):
        super().__init__()
        self.action_space = action_space.n
        print(self.action_space)
        # CNN for cropped images
        obs_space_crop = torch.squeeze(obs_space_crop)
        
        self.conv_1 = nn.Sequential(
            nn.Conv2d(obs_space_crop.shape[0], 16, 3, stride=1, padding=1), #channels, outgoing_filters, kernerl_size, stride/no_of_frames
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),   
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
        )

        self.input_dim_crop = self.conv_1(torch.zeros(1, *obs_space_crop.shape)).view(1, -1).size(1)
        print('crop output: ',self.input_dim_crop)
        
        # CNN for whole images
        
        obs_space_whole = torch.squeeze(obs_space_whole)

        self.conv_2 = nn.Sequential(
            nn.Conv2d(obs_space_whole.shape[0], 16, 3, stride=4, padding=1), #channels, outgoing_filters, kernerl_size, stride/no_of_frames
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),   
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
        )

        self.input_dim_whole = self.conv_2(torch.zeros(1, *obs_space_whole.shape)).view(1, -1).size(1)
        print('whole output: ', self.input_dim_whole)
        
        
        # CNN for Inventory
        
        obs_space_inv = torch.squeeze(obs_space_inv, 1)

        self.conv_3 = nn.Sequential(
            nn.Conv2d(obs_space_inv.shape[0], 16, 3, stride=4, padding=1), #channels, outgoing_filters, kernerl_size, stride/no_of_frames
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),   
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, stride=1, padding=1),
            nn.ReLU(),
        )

        self.input_dim_inv = self.conv_3(torch.zeros(1, *obs_space_inv.shape)).view(1, -1).size(1)
        print('inventory output: ', self.input_dim_inv)
        
        # MLP_1
        
        self.input_dim_mlp = obs_space_stats.shape[0]
        self.output_dim_mlp = 512
        self.mlp_1 = nn.Sequential(
            nn.Linear(self.input_dim_mlp, 512),
            nn.ReLU(),
            nn.Linear(512, self.output_dim_mlp)
        )
        
        # Fully Connected 2nd Layer
        
        print(self.input_dim_crop+self.input_dim_whole+self.input_dim_inv+self.output_dim_mlp)
        self.fc1_output = 1024
        self.fc1 = nn.Sequential(
            nn.Linear(self.input_dim_crop+self.input_dim_whole+self.input_dim_inv+self.output_dim_mlp, 2048),
            nn.ReLU(),
            nn.Linear(2048, self.fc1_output)
        )
        
        # LSTM
        
        self.lstm_size = self.fc1_output
        self.num_layers = 2
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(self.fc1_output, 512),
            nn.ReLU(),
            nn.Linear(512, self.action_space),
            nn.Softmax(dim=-1),
        )

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, crop, whole, inv, stats):
        conv_1 = self.conv_1(crop).view(-1)
        conv_2 = self.conv_2(whole).view(-1)
        conv_3 = self.conv_3(inv).view(-1)
        mlp_1 = self.mlp_1(stats).view(-1)
        x = torch.cat([conv_1, conv_2, conv_3, mlp_1])
        x = self.fc1(x)
        #print(x.unsqueeze(0).shape)
        #x, new_state = self.lstm(x.unsqueeze(0))
        x = self.fc2(x)
        return x
    
full_model = combined(crop_state, whole_state, inv_state, stats_state, env.action_space)
x_4 = full_model.forward(crop_state.to(full_model.device),whole_state.to(full_model.device),inv_state.to(full_model.device),stats_state.to(full_model.device))
print(torch.argmax(x_4))

78
crop output:  1296
whole output:  1920
inventory output:  1120
4848
tensor(75, device='cuda:0')
